In [ ]:
import os
import sys
import glob
import pandas as pd

import spatialdata as sd
from spatialdata_io import xenium
import spatialdata_io
import spatialdata_plot

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq




In [ ]:

saveFolder="/projects/b1146/BharatLab/Yuanqing/Projects/Spatial/IPF"
FigOut="/projects/b1042/Yuanqing/Sptial/data/output"
folderPath="/projects/b1146/BharatLab/20240530__190352__20240530_X1_Bharat"
HEpath='/projects/b1146/BharatLab/20240530__190352__20240530_X1_Bharat/allHE'
sampleFile=glob.glob(f'{folderPath}/*__*')
sampleFile

In [ ]:
# output-XETG00190__0025154__Bharat_hLung_1__20240530__190454          IPF3R
# output-XETG00190__0025154__Bharat_hLung_2__20240530__190455          IPF5
# output-XETG00190__0025154__Bharat_hLung_3__20240530__190455          IPF4
# output-XETG00190__0025154__Bharat_hLung_4__20240530__190455 (donor)  Donor2
# output-XETG00190__0025157__Bharat_hLung_1__20240530__190455          IPF3L
# output-XETG00190__0025157__Bharat_hLung_2__20240530__190455          IPF2
# output-XETG00190__0025157__Bharat_hLung_3__20240530__190455          IPF1
# output-XETG00190__0025157__Bharat_hLung_4__20240530__190455 (donor)  Donor1


###Olddisease=["IPF3R", "IPF5", "IPF4", "Donor2", "IPF3L", "IPF2", "IPF1", "Donor1"]
disease=["IPF4","IPF1", "IPF3L", "IPF2", "Donor1", "IPF5", "IPF3R", "Donor2"]
zarr=[saveFolder+f'/Zarr/{disease[i]}.zarr' for i in range(len(disease))]
zarr
zarr2=[saveFolder+f'/Zarr_withCT/{disease[i]}.zarr' for i in range(len(disease))]
pdata=pd.DataFrame({'sam':sampleFile, 'disease':disease, 'Zarr':zarr, 'Zarr2':zarr2})
pdata


In [ ]:

for i in range(pdata.shape[0]):
    print(f"{pdata.loc[i,'disease']}")
    temp = xenium(pdata.loc[i,'sam'])
    
    if pdata.loc[i,'disease']=="IPF3L":
        he_img = spatialdata_io.xenium_aligned_image(image_path=f"{HEpath}/IPF3LL.ome.tif",
                                             alignment_file=f"{HEpath}/IPF3LL_matrix.csv")
        he_img2 = spatialdata_io.xenium_aligned_image(image_path=f"{HEpath}/IPF3LR.ome.tif",
                                             alignment_file=f"{HEpath}/IPF3LR_matrix.csv")
        temp.images["HE"] = he_img
        temp.images["HER"] = he_img2
    else:
        # Load the saved H&E image into SpatialData
        he_img = spatialdata_io.xenium_aligned_image(image_path=f"{HEpath}/{pdata.loc[i,'disease']}.ome.tif",
                                             alignment_file=f"{HEpath}/{pdata.loc[i,'disease']}_matrix.csv")
        # Add it to the SpatialData object
        temp.images["HE"] = he_img    #del sdata.images["he_IPF5"]

    temp.write(pdata.loc[i,'Zarr'])
    
    

In [ ]:
metaData=pd.read_csv("/projects/b1146/BharatLab/Yuanqing/Projects/Spatial/IPF/XeniumMeta_ILD_B_Paper.csv")
metaData.head(5)



In [ ]:
pdata

In [ ]:
colnameMeta=['SamCellID','CellID','CellType',"nCount_niche6","nFeature_niche6","niches6",
             "nCount_niche7","nFeature_niche7", "niches7", 
             "nCount_niche8", "nFeature_niche8", "niches8", 
             "nCount_niche9", "nFeature_niche9", "niches9", 
             "nCount_niche10", "nFeature_niche10", "niches10", 
             "nCount_niche11", "nFeature_niche11", "niches11", 
             "nCount_niche12", "nFeature_niche12", "niches12", 
             "nCount_niche13", "nFeature_niche13", "niches13", 
             "nCount_niche14", "nFeature_niche14", "niches14", 
             "nCount_niche15", "nFeature_niche15", "niches15"]

def addCT_Leiden(sddata,metaData,Sample,minCount=1,minCell=1):
    metaTemp=metaData.loc[metaData['orig.ident']==Sample][['orig.ident']+colnameMeta]
    sdCell=sddata.tables["table"].obs[['cell_id','cell_labels']]
    sdCell.index.name=None
    annCell=metaTemp[colnameMeta]
    annCell.rename(columns={'CellID':'cell_id'},inplace=True)
    left_joined_cell = pd.merge(sdCell, annCell, on="cell_id", how="left")
    if not (sdCell['cell_id'].reset_index(drop=True) == left_joined_cell['cell_id'].reset_index(drop=True)).all():
        raise Exception("error")    
        
    left_joined_cell.index.name=None
    sddata.tables["table"].obs['CellType']=list(left_joined_cell['CellType'].fillna("unknown"))
    
    sddata.tables["table"].obs['nCount_niche6']=list(left_joined_cell['nCount_niche6'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche6']=list(left_joined_cell['nFeature_niche6'].fillna(0))
    sddata.tables["table"].obs['niches6']=list(left_joined_cell['niches6'].fillna(99))
    
    sddata.tables["table"].obs['nCount_niche7']=list(left_joined_cell['nCount_niche7'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche7']=list(left_joined_cell['nFeature_niche7'].fillna(0))
    sddata.tables["table"].obs['niches7']=list(left_joined_cell['niches7'].fillna(99))
    
    sddata.tables["table"].obs['nCount_niche8']=list(left_joined_cell['nCount_niche8'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche8']=list(left_joined_cell['nFeature_niche8'].fillna(0))
    sddata.tables["table"].obs['niches8']=list(left_joined_cell['niches8'].fillna(99))
    
    sddata.tables["table"].obs['nCount_niche9']=list(left_joined_cell['nCount_niche9'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche9']=list(left_joined_cell['nFeature_niche9'].fillna(0))
    sddata.tables["table"].obs['niches9']=list(left_joined_cell['niches9'].fillna(99))
    
    sddata.tables["table"].obs['nCount_niche10']=list(left_joined_cell['nCount_niche10'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche10']=list(left_joined_cell['nFeature_niche10'].fillna(0))
    sddata.tables["table"].obs['niches10']=list(left_joined_cell['niches10'].fillna(99))
    
    sddata.tables["table"].obs['nCount_niche11']=list(left_joined_cell['nCount_niche11'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche11']=list(left_joined_cell['nFeature_niche11'].fillna(0))
    sddata.tables["table"].obs['niches11']=list(left_joined_cell['niches11'].fillna(99))
    
    sddata.tables["table"].obs['nCount_niche12']=list(left_joined_cell['nCount_niche12'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche12']=list(left_joined_cell['nFeature_niche12'].fillna(0))
    sddata.tables["table"].obs['niches12']=list(left_joined_cell['niches12'].fillna(99))
    
    sddata.tables["table"].obs['nCount_niche13']=list(left_joined_cell['nCount_niche13'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche13']=list(left_joined_cell['nFeature_niche13'].fillna(0))
    sddata.tables["table"].obs['niches13']=list(left_joined_cell['niches13'].fillna(99))
    
    sddata.tables["table"].obs['nCount_niche14']=list(left_joined_cell['nCount_niche14'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche14']=list(left_joined_cell['nFeature_niche14'].fillna(0))
    sddata.tables["table"].obs['niches14']=list(left_joined_cell['niches14'].fillna(99))
    
    sddata.tables["table"].obs['nCount_niche15']=list(left_joined_cell['nCount_niche15'].fillna(0))
    sddata.tables["table"].obs['nFeature_niche15']=list(left_joined_cell['nFeature_niche15'].fillna(0))
    sddata.tables["table"].obs['niches15']=list(left_joined_cell['niches15'].fillna(99))
#     sc.pp.filter_cells(sddata.tables["table"], min_counts=minCount)
#     sc.pp.filter_genes(sddata.tables["table"], min_cells=minCell)
#     sddata.tables["table"].layers["counts"] = sddata.tables["table"].X.copy()
#     sc.pp.normalize_total(sddata.tables["table"], target_sum=1e4, inplace=True)
#     sc.pp.log1p(sddata.tables["table"])
#     sc.pp.pca(sddata.tables["table"])
#     sc.pp.neighbors(sddata.tables["table"])
#     sc.tl.umap(sddata.tables["table"])
#     sc.tl.leiden(sddata.tables["table"])
    
    return sddata

allData=list(pdata['disease'])
allData

for i in range(len(allData)):
    print(f"Analyze {allData[i]}")
    TempSdata = sd.read_zarr(pdata.loc[i,'Zarr'])
    TempSdataFinal=addCT_Leiden(sddata=TempSdata,
                                metaData=metaData,
                                Sample=allData[i],
                                minCount=1,
                                minCell=1)
    TempSdataFinal.write(pdata.loc[i,'Zarr2'])
    

In [ ]:
IPF2=sd.read_zarr(pdata.loc[3,'Zarr2']) 
IPF2

In [ ]:
from spatialdata import bounding_box_query
def crop0(x):
    return bounding_box_query(
        x,
        min_coordinate=[16000, 20000],
        max_coordinate=[18000, 22000],
        axes=("x", "y"),
        target_coordinate_system="global",
    )

In [ ]:
crop0(IPF2).pl.render_images("HE").pl.render_shapes("cell_circles",
    color='CD79A',
    size=0.2
).pl.show(coordinate_systems="global", 
          figsize=(10, 10))